In [13]:
import numpy
from tkinter import *

def onlinum(string, mode='num'):
    nums = ['1','2','3','4','5','6','7','8','9','0','+', '-', '=']
    out = ''
    if mode == 'num':
        for i in string:
            if i in nums:
                if i == '+':
                    out = out + '|'
                elif i == '-':
                    out = out + '|-'
                else:
                    out = out + i
    elif mode == 'sym':
        nums = nums + ['(',')','|','^','*']
        for i in string:
            if i == '=':
                break
            elif i not in nums:
                out = out + i
    else:
        for i in string:
            if i == '+' or i == '-':
                out = out + '|'
            elif i == '=':
                break
            elif i not in nums:
                out = out + i
    out = out.replace(' ', '')
    return out


def noline(notLine, dict_line):
    for i in dict_line.keys():
        notLine = notLine.replace(i, str('*' + str(dict_line[i])))
    notLine_sym = onlinum(notLine, mode='sym')
    ur = notLine.replace(' ', '').split('=')
    if ur[1][0] == '-':
        comp_b = '+' + str(ur[1][1:])
    else:
        comp_b = '-' + str(ur[1])
    components = str(ur[0]) + comp_b
    components = components.replace('+', '|').replace('-', '|-')
    if components[0] == '|': components = components[1:]
    components = components.split("|")
    ur = ['', []]
    for i in components:
        if len(onlinum(i, mode='sym')) > 0:
            ur[0] = i
        else:
            if i[0] == '-':
                i = i[1:]
            else:
                i = '-' + str(i)
            ur[1].append(i)
    rt_num = 0
    lt_num = int(str(onlinum(ur[0])).replace('|', ''))
    ur[0] = onlinum(ur[0], mode='sym')
    ur.append(int(lt_num))
    ot = 0
    for i in ur[1]:
        out_b = 1
        if i.find('*') > 0:
            i = i.split('*')
            out_c = 1
            for a in i:
                if a.find('^') > 0:
                    a = a.split('^')
                    out_c = int(a[0]) ** int(a[1])
                else:
                    out_c = a
                out_b = out_b * int(out_c)
            ot = ot + out_b
        else:
            ot = ot + int(i)
    ot = int(ot) / ur[2]
    dict_line[str(ur[0])] = ot
    label2['text'] = label2['text'] + str(dict_line) + '\n'
    return dict_line



def matrix(lines):
    dict_line = {}
    matrix_a = []
    vector = []
    symbols = str(onlinum(lines[0], mode='nn')).split('|')
    for i in lines:
        if len(i) < 2: continue
        i = onlinum(i)
        i = i.replace('=|', '=')
        mtx, vt = i.split('=')
        vek_a = mtx.split('|')
        ovek_a = []
        for a in vek_a:
            ovek_a.append(float(a))
        matrix_a.append(ovek_a)
        vector.append(float(vt))
    A = numpy.array(matrix_a)
    B = numpy.array(vector)
    Aa = numpy.linalg.inv(A)
    X = B * Aa
    out = []
    for i in X:
        tmp_o = 0
        for a in i:
            tmp_o = tmp_o + a
        if tmp_o % 1 > 0.5:
            tmp_o = int(tmp_o) + 1
        else:
            tmp_o = int(tmp_o)
        out.append(tmp_o)
    dict_out = {}
    for i in range(len(out)):
        dict_out[symbols[i]] = out[i]
    return dict_out


def readFormFile():
    label2['text'] = ''
    filename = e.get()
    
    #try:
    formula = open(filename, 'r').read().replace('\n',';').replace('\r', '').replace(' ', '')
    while formula.find(';;') > 0:
        formula = formula.replace(';;', ';')
    formula = formula.split('{')[1:]
    for i in formula:
        i = i.replace('}', '')
        label2['text'] = label2['text'] + i + '\n'
        evalEqSys(i)
    #except:
        #print('Error')


def readForm():
    label2['text'] = ''
    formula = e.get()
    formula = formula.strip()
    formula = formula.split('{')[1:]
    for i in formula:
        i = i.replace('}', '')
        label2['text'] = label2['text'] + i + '\n'
        evalEqSys(i)


def evalEqSys(eq):
    eq = eq.split(';')
    notLine = ''
    line = []
    for i in eq:
        if i.find('^') > 0:
            notLine = i
        else:
            line.append(i)
    noline(notLine, matrix(line))

    
def start(a):
    sel = e.get()
    if sel == '1':
        b["command"] = readForm
        b["text"] = 'Решить'
        b.place(relx=.9, rely=.0)
        b['height'] = 30
        b['width'] = 20
        label2['text'] = "Введите систему уравнений: "
        tk.title("Введите систему уравнений: ")
        label2.place(relx=.0, rely=.0)
    if sel == '2':
        b["command"] = readFormFile
        label2['text'] = "Введите имя файла c системой уравнений:"
    if sel == '0':
        label2['text'] = "Программа закончила свою работу"
        

def callback():
    a = e.get()
    start(a)
        

tk = Tk()
tk['bg'] = 'white'
tk.title('Калькулятор систем')
tk.wm_attributes('-alpha', 1)
tk.geometry('640x480')
e = Entry(tk)
e.pack()
e.focus_set()
b = Button(tk, text = "OK",heigh = 5,width = 5, command = callback)
b.pack()
poetry = "Получить формулу из:\n 1. Строки ввода\n 2. Файла\n 0. Выход"
label2 = Label(text=poetry, justify=LEFT)
label2.place(relx=.01, rely=.01)
mainloop()